# Etapa 1

- Treinamento inicial do modelo YoloV11
 

In [1]:
import os
import torch
from pycocotools.coco import COCO
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import shutil

In [2]:
# Diretórios
DATA_DIR = "E:/COCO-Dataset/"
os.makedirs(DATA_DIR, exist_ok=True)

img_dir = os.path.join("E:/COCO-Dataset/", "val2017/val2017/")
ann_file = os.path.join("E:/COCO-Dataset/","annotations2017/annotations2017/instances_val2017.json")

# Carregar anotações
coco = COCO(ann_file)
img_ids = coco.getImgIds()

categories = coco.loadCats(coco.getCatIds())
categories.sort(key=lambda x: x['id'])  # Ordenar por ID original

# Mapear ID original → ID sequencial (0-79)
coco_id_to_yolo_id = {cat['id']: idx for idx, cat in enumerate(categories)}
print("Mapeamento de IDs:", coco_id_to_yolo_id)


loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
Mapeamento de IDs: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 13: 11, 14: 12, 15: 13, 16: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19, 22: 20, 23: 21, 24: 22, 25: 23, 27: 24, 28: 25, 31: 26, 32: 27, 33: 28, 34: 29, 35: 30, 36: 31, 37: 32, 38: 33, 39: 34, 40: 35, 41: 36, 42: 37, 43: 38, 44: 39, 46: 40, 47: 41, 48: 42, 49: 43, 50: 44, 51: 45, 52: 46, 53: 47, 54: 48, 55: 49, 56: 50, 57: 51, 58: 52, 59: 53, 60: 54, 61: 55, 62: 56, 63: 57, 64: 58, 65: 59, 67: 60, 70: 61, 72: 62, 73: 63, 74: 64, 75: 65, 76: 66, 77: 67, 78: 68, 79: 69, 80: 70, 81: 71, 82: 72, 84: 73, 85: 74, 86: 75, 87: 76, 88: 77, 89: 78, 90: 79}


In [3]:
# Listar todas as categorias com IDs originais e YOLO
for cat in categories:
    print(f"COCO ID: {cat['id']} → YOLO ID: {coco_id_to_yolo_id[cat['id']]} | Nome: {cat['name']}")

COCO ID: 1 → YOLO ID: 0 | Nome: person
COCO ID: 2 → YOLO ID: 1 | Nome: bicycle
COCO ID: 3 → YOLO ID: 2 | Nome: car
COCO ID: 4 → YOLO ID: 3 | Nome: motorcycle
COCO ID: 5 → YOLO ID: 4 | Nome: airplane
COCO ID: 6 → YOLO ID: 5 | Nome: bus
COCO ID: 7 → YOLO ID: 6 | Nome: train
COCO ID: 8 → YOLO ID: 7 | Nome: truck
COCO ID: 9 → YOLO ID: 8 | Nome: boat
COCO ID: 10 → YOLO ID: 9 | Nome: traffic light
COCO ID: 11 → YOLO ID: 10 | Nome: fire hydrant
COCO ID: 13 → YOLO ID: 11 | Nome: stop sign
COCO ID: 14 → YOLO ID: 12 | Nome: parking meter
COCO ID: 15 → YOLO ID: 13 | Nome: bench
COCO ID: 16 → YOLO ID: 14 | Nome: bird
COCO ID: 17 → YOLO ID: 15 | Nome: cat
COCO ID: 18 → YOLO ID: 16 | Nome: dog
COCO ID: 19 → YOLO ID: 17 | Nome: horse
COCO ID: 20 → YOLO ID: 18 | Nome: sheep
COCO ID: 21 → YOLO ID: 19 | Nome: cow
COCO ID: 22 → YOLO ID: 20 | Nome: elephant
COCO ID: 23 → YOLO ID: 21 | Nome: bear
COCO ID: 24 → YOLO ID: 22 | Nome: zebra
COCO ID: 25 → YOLO ID: 23 | Nome: giraffe
COCO ID: 27 → YOLO ID: 24 | N

In [4]:
# Dividir dados em treino (20%) e validação (80%)
train_ids, val_ids = train_test_split(img_ids, test_size=0.8, random_state=42)


In [5]:
# Diretórios de saída
train_img_dir = os.path.join(DATA_DIR, "val2017/train/images")
train_label_dir = os.path.join(DATA_DIR, "val2017/train/labels")
val_img_dir = os.path.join(DATA_DIR, "val2017/val/images")
val_label_dir = os.path.join(DATA_DIR, "val2017/val/labels")

# Criar diretórios
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)



In [6]:

def process_split(img_ids, target_img_dir, target_label_dir):
    for img_id in tqdm(img_ids):
        img_info = coco.loadImgs(img_id)[0]
        img_file = img_info["file_name"]
        img_path = os.path.join(img_dir, img_file)
        
        # Mover imagem
        shutil.copy(img_path, os.path.join(target_img_dir, img_file))
        
        # Gerar rótulo YOLO
        label_path = os.path.join(target_label_dir, img_file.replace(".jpg", ".txt"))
        ann_ids = coco.getAnnIds(imgIds=img_id)
        annotations = coco.loadAnns(ann_ids)
        
        with open(label_path, "w") as f:
            for ann in annotations:
                if "bbox" not in ann or ann["area"] <= 0:
                    continue
                
                # Converter ID COCO para YOLO
                yolo_class_id = coco_id_to_yolo_id.get(ann['category_id'], -1)
                if yolo_class_id == -1:  # Pular classes não mapeadas
                    continue
                
                # Converter bbox
                x, y, w, h = ann["bbox"]
                img_width = img_info["width"]
                img_height = img_info["height"]
                
                x_center = (x + w / 2) / img_width
                y_center = (y + h / 2) / img_height
                w_norm = w / img_width
                h_norm = h / img_height
                
                f.write(f"{yolo_class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

In [7]:
# Processar split de treino
process_split(train_ids, train_img_dir, train_label_dir)

# Processar split de validação
process_split(val_ids, val_img_dir, val_label_dir)

100%|██████████| 4000/4000 [00:33<00:00, 117.73it/s]


In [8]:
from ultralytics import YOLO

# Configuração do modelo (supondo que 'yolov11x' é a versão completa)
model = YOLO("yolo11x.pt")    # Se existir um weights pré-treinado

# Hiperparâmetros customizados (opcional)
model.args = {
    'epochs': 50,
    'batch': 8,
    'imgsz': 480,
    'optimizer': 'AdamW',
    'lr0': 0.001,
    'data': 'coco.yaml',
    'device' : 0,
}

In [ ]:

# Treinar
results = model.train(
    data="coco.yaml",
    epochs=50,
    imgsz=480,
    batch=8,
    cache=False,
    name="yolov11-initial"
)

New https://pypi.org/project/ultralytics/8.3.137 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.135  Python-3.13.2 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11-initial, nbs=64, nms=False, opset=None, optimize=Fal

train: Scanning E:\COCO-Dataset\val2017\train\labels... 1000 images, 9 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:01<00:00, 567.03it/s]

train: New cache created: E:\COCO-Dataset\val2017\train\labels.cache


val: Fast image access  (ping: 0.10.0 ms, read: 20.24.9 MB/s, size: 183.2 KB)


val: Scanning E:\COCO-Dataset\val2017\val\labels... 4000 images, 39 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:07<00:00, 516.12it/s]


...

      50/50      5.46G     0.5475     0.3345     0.8898         51        480: 100%|██████████| 125/125 [00:47<00:00,  2.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 250/250 [00:58<00:00,  4.31it/s]

                   all       4000      29401      0.605      0.436      0.461      0.325



50 epochs completed in 1.613 hours.
Optimizer stripped from runs\detect\yolov11-initial\weights\last.pt, 114.5MB
Optimizer stripped from runs\detect\yolov11-initial\weights\best.pt, 114.5MB

Validating runs\detect\yolov11-initial\weights\best.pt...
Ultralytics 8.3.135  Python-3.13.2 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 8192MiB)
YOLO11x summary (fused): 190 layers, 56,919,424 parameters, 0 gradients, 194.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 250/250 [00:56<00:00,  4.43it/s]


                   all       4000      29401      0.692      0.592      0.636      0.471
                person       2167       8916       0.87      0.684       0.81      0.587
               bicycle        118        250      0.671      0.588      0.634      0.374
                   car        432       1587      0.779      0.585      0.681      0.455
            motorcycle        132        311      0.722      0.733      0.762      0.519
              airplane         80        117      0.763      0.838      0.857      0.687
                   bus        153        239      0.735      0.753      0.802       0.68
                 train        134        160       0.95       0.83      0.909      0.728
                 truck        198        318      0.382      0.657      0.532      0.398
                  boat         97        341      0.626      0.505      0.544      0.305
         traffic light        146        475      0.664      0.469       0.52        0.3
          fire hydran